In [1]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px

# Load the preprocessed data
wine_df = pd.read_csv('preprocessed_wine_data.csv')

external_stylesheets = [
    {
        "href": "assets/style.css",
            # "https://fonts.googleapis.com/css2?"
            # "family=Lato:wght@400;700&display=swap"
        
        "rel": "stylesheet",
    },
]

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "The Story of Wine: From Grapes to Glass!"

# Layout of the dashboard
app.layout = html.Div([
    # Create a flex container for the H1 and the image
    html.Div(
        style={'display': 'flex', 'alignItems': 'center'},  # Align items in the center vertically
        children=[
            html.H1("Wine Statistics Dashboard", style={'marginRight': '20px'}),  # Add some margin to the right
            html.Img(src='assets/wine.jpg', className='header-emoji', style={'width': '100px', 'height': 'auto'})  # Adjust size as needed
        ]
    ),
    
    # Dropdown for selecting country
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in wine_df['Country'].unique()],
        value=wine_df['Country'].unique()[0],  # Default value
        multi=False
    ),
    
    # Container for the graphs
    html.Div(
        style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'space-between'},
        children=[
            # Graph for price distribution
            html.Div(dcc.Graph(id='price-distribution'), style={'flex': '1 1 30%', 'margin': '10px'}),
            
            # Graph for average price by country
            html.Div(dcc.Graph(id='avg-price-country'), style={'flex': '1 1 30%', 'margin': '10px'}),
            
            # Graph for food pairings
            html.Div(dcc.Graph(id='food-pairings'), style={'flex': '1 1 30%', 'margin': '10px'}),
            
            # Graph for price vs. rating
            html.Div(dcc.Graph(id='price-vs-rating'), style={'flex': '1 1 30%', 'margin': '10px'}),
            
            # Placeholder for the fifth graph
            html.Div(dcc.Graph(id='fifth-graph'), style={'flex': '1 1 30%', 'margin': '10px'}),
            
            # Placeholder for the sixth graph
            html.Div(dcc.Graph(id='sixth-graph'), style={'flex': '1 1 30%', 'margin': '10px'}),
        ]
    )
])

# Callback to update the price distribution graph
@app.callback(
    dash.dependencies.Output('price-distribution', 'figure'),
    [dash.dependencies.Input('country-dropdown', 'value')]
)
def update_price_distribution(selected_country):
    filtered_df = wine_df[wine_df['Country'] == selected_country]
    fig = px.box(filtered_df, y='Price', title=f'Price Distribution for {selected_country}')
    return fig

# Callback to update average price by country graph
@app.callback(
    dash.dependencies.Output('avg-price-country', 'figure'),
    [dash.dependencies.Input('country-dropdown', 'value')]  # Add input here
)
def update_avg_price_country(selected_country):
    avg_price = wine_df.groupby('Country')['Price'].mean().reset_index()
    fig = px.bar(avg_price, x='Country', y='Price', title='Average Price by Country')
    return fig

# Callback to update food pairings graph
@app.callback(
    dash.dependencies.Output('food-pairings', 'figure'),
    [dash.dependencies.Input('country-dropdown', 'value')]  # Add input here
)
def update_food_pairings(selected_country):
    filtered_df = wine_df[wine_df['Country'] == selected_country]
    food_columns = wine_df.columns[12:]  # Assuming food pairing columns start from the 13th column
    food_counts = wine_df[food_columns].sum().reset_index()
    food_counts.columns = ['Food', 'Count']
    fig = px.bar(food_counts, x='Food', y='Count', title=f'Food Pairings Count for {selected_country}')
    return fig

# Callback to update price vs rating graph
@app.callback(
    dash.dependencies.Output('price-vs-rating', 'figure'),
    [dash.dependencies.Input('country-dropdown', 'value')]  # Add input here
)
def update_price_vs_rating(selected_country):
    filtered_df = wine_df[wine_df['Country'] == selected_country]
    filtered_df = filtered_df.dropna(subset=['Price', 'Rating'])
    fig = px.scatter(filtered_df, x='Price', y='Rating', title='Price vs. Rating', trendline='ols')
    return fig

# Placeholder callbacks for the fifth and sixth graphs
# @app.callback(
#     dash.dependencies.Output('fifth-graph', 'figure'),
#     [dash.dependencies.Input('country-dropdown', 'value')]
# )
# def update_fifth_graph(selected_country):
#     filtered_df = wine_df[wine_df['Country'] == selected_country]
#     filtered_df = filtered_df.dropna(subset=['Alcohol content', 'Rating'])  # Drop rows with NaN values in Alcohol or Rating
#     fig = px.scatter(filtered_df, x='Alcohol content', y='Rating', title='Alcohol Content vs. Rating', trendline='ols')
#     return fig

# Callback to update the fifth graph (Alcohol vs Rating)
@app.callback(
    dash.dependencies.Output('fifth-graph', 'figure'),
    [dash.dependencies.Input('country-dropdown', 'value')]
)
def update_fifth_graph(selected_country):
    filtered_df = wine_df[wine_df['Country'] == selected_country]
    filtered_df = filtered_df.dropna(subset=['Alcohol content', 'Rating'])  # Drop rows with NaN values in Alcohol or Rating
    
    # Group by Rating and calculate the average Alcohol content
    avg_alcohol_by_rating = filtered_df.groupby('Rating')['Alcohol content'].mean().reset_index()
    
    # Create a line graph
    fig = px.line(avg_alcohol_by_rating, x='Rating', y='Alcohol content', title='Average Alcohol Content by Rating')
    return fig

@app.callback(
    dash.dependencies.Output('sixth-graph', 'figure'),
    [dash .dependencies.Input('country-dropdown', 'value')]
)
def update_sixth_graph(selected_country):
    # Placeholder logic for the sixth graph
   # fig = px.line(wine_df, x='AnotherColumn', y='Price', title='Sixth Graph Placeholder')
   # return fig
   return 0

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)